In [1]:
import re
from collections import defaultdict, namedtuple

from lazyme import per_section

NMTMeta = namedtuple('NMTMeta', 'name website who comments')

def get_name_website(s):
    try:
        caught = re.search(r"\|?(.*)?\[(.*)\]\((.*)\)(.*)?", s)
        if s.startswith('###'):
            name = caught.group(2)
        else:
            name = ' '.join([caught.group(1), caught.group(2), caught.group(4)]).strip()
        website = caught.group(3)
    except AttributeError:
        name = re.search(r"(###)?\s(.*)", s).group(2)
        website = None
    return name, website


nmt_list = defaultdict(list)

with open('README.md') as fin:
    for section in list(per_section(fin)):
        if '| Software | Who | Comments |' in section or section[0] == '### Other':
            header_name, header_website = get_name_website(section[0])
            nmt_type = (header_name, header_website)
            for row in section[3:]:
                first_column = row.split(' | ')[0]
                name, website  = get_name_website(first_column)
                who = row.split(' | ')[1]
                comments = row.split(' | ')[2]
                nmt_tool = NMTMeta._make([name, website, who, comments])
                nmt_list[nmt_type].append(nmt_tool)
                

In [2]:
def loop_thru_nmt(nmt_list):
    for nmt_type in nmt_list:
        for nmt_tool in nmt_list[nmt_type]:
            yield nmt_tool

In [3]:
num_nmt, num_seq2seq, num_dl4mt = 0, 0, 0
num_derivedfrom_dl4mt = 0

for nmt_tool in loop_thru_nmt(nmt_list):
    name, website, who, comments = list(nmt_tool)
    if 'nmt' in name:
        num_nmt += 1
    if 'seq2seq' in name:
        num_seq2seq += 1
    if 'dl4mt' in name:
        num_dl4mt += 1
        num_derivedfrom_dl4mt += 1
    if 'dl4mt' in comments:
        num_derivedfrom_dl4mt += 1

In [4]:
template = """
## Project Stats
* **{nmt}** project names include *nmt*
* **{seq2seq}** project names include *seq2seq*
* **{dl4mt}** project names include *dl4mt*
* **{derived}** projects are derived from Kyunghyun Cho's tutorial at the 2015 [DL4MT winter school](http://dl4mt.computing.dcu.ie)
"""

print (template.format(nmt=num_nmt, seq2seq=num_seq2seq, dl4mt=num_dl4mt, derived=num_derivedfrom_dl4mt))


## Project Stats
* **5** project names include *nmt*
* **5** project names include *seq2seq*
* **2** project names include *dl4mt*
* **3** projects are derived from Kyunghyun Cho's tutorial at the 2015 [DL4MT winter school](http://dl4mt.computing.dcu.ie)

